# image_alignment

In [4]:
import numpy as np
import cv2
import os
import glob

def image_alignment(template,image, sift_thres = 0.7, ransac_thres = 5.0):
    """
    align image to the template image with SIFT.
    args:
        template: template image
        image: target image to fit to template image
    return:
        aligned): image after alignment
    """
    # convert to gray
    tmp = cv2.cvtColor(template,cv2.COLOR_BGR2GRAY)
    img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    # compute keypoints and descriptors
    sift = cv2.xfeatures2d.SIFT_create()
    img_kp, img_dsc = sift.detectAndCompute(img,None)
    tmp_kp, tmp_dsc = sift.detectAndCompute(tmp,None)
    
    # find where are good matches
    matcher = cv2.BFMatcher_create(normType=cv2.NORM_L2,crossCheck=False)
    matches = matcher.knnMatch(img_dsc, tmp_dsc,k=2)
    good_matches = []
    for m1,m2 in matches:
        if m1.distance < sift_thres*m2.distance:
            good_matches.append(m1)
            
    # apply homography
    img_pts = np.float32([img_kp[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
    tmp_pts = np.float32([tmp_kp[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)

    homo, mask = cv2.findHomography(img_pts, tmp_pts, cv2.RANSAC, ransac_thres)

    h,w,_ = template.shape
    image_aligned = cv2.warpPerspective(image, homo, (w,h))
    
    return image_aligned


In [5]:
images_path = 'dataset/images'
results_path = 'dataset/aligned'
if not os.path.isdir(results_path):
        os.makedirs(results_path)
datasets = ['boxes', 'cotton']

for dataset in datasets:
    if not os.path.isdir(os.path.join(results_path, dataset)):
        os.makedirs(os.path.join(results_path, dataset))
        
    images = sorted(glob.glob(os.path.join(images_path, dataset, '*')))
    template = cv2.imread(images[0])
    cv2.imwrite(images[0].replace('images', 'aligned'), template)

    for image in images[1:]:
        template = image_alignment(template, cv2.imread(image))
        cv2.imwrite(image.replace('images', 'aligned'), template)

In [ ]:
results_path = 'dataset/errormap'
if not os.path.isdir(results_path):
        os.makedirs(results_path)
datasets = ['boxes', 'cotton']

for dataset in datasets:
    if not os.path.isdir(os.path.join(results_path, dataset)):
        os.makedirs(os.path.join(results_path, dataset))
        
    images = sorted(glob.glob(os.path.join(images_path, dataset, '*')))
    template = cv2.imread(images[0])
    cv2.imwrite(images[0].replace(images_path, results_path), template)

    for image in images[1:]:
        error=[]
        img = cv2.imread(image)
        error.append(img-template)
        img_aligned = image_alignment(template, img)
        error.append(img_aligned-template)
        template = img_aligned
        
        error = np.concatenate(error)
        cv2.imwrite(image.replace(images_path, results_path), error)

# initial depth from focus measure

In [ ]:
def get_focus(img):
    """
    compute Tenenbaum focus measure
    args:
        image
    return:
        tenenbaum: computed tenenbaum focus measure from the input image
    """
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (3,3), 0)
    img_x = cv2.Sobel(img,cv2.CV_32F,1,0,ksize=3)
    img_y = cv2.Sobel(img,cv2.CV_32F,0,1,ksize=3)
    tenenbaum = img_x*img_x + img_y*img_y
    return tenenbaum

# all-in-focus image

In [ ]:
images_path = 'dataset/aligned'
results_path = 'dataset/focus'
if not os.path.isdir(results_path):
        os.makedirs(results_path)
datasets = ['boxes', 'cotton']

for dataset in datasets:
    if not os.path.isdir(os.path.join(results_path, dataset)):
        os.makedirs(os.path.join(results_path, dataset))
    image_files = sorted(glob.glob(os.path.join(images_path, dataset, '*')))
    
    imgs = []
    focus = []    
    # get focus maps
    for f in image_files:
        img = cv2.imread(f)
        img_focus = get_focus(img)
        cv2.imwrite(f.replace('aligned', 'focus'), img_focus)
        
        imgs.append(img)
        focus.append(img_focus)
            
    # initial depth map
    focus = np.array(focus)
    depth_map = np.argmax(focus, axis=0)
    cv2.imwrite(os.path.join(results_path, 'depthmap_%s.png'%(dataset)), depth_map)
    print(depth_map.shape)
    
    # retrieve focused image
    focused = np.zeros(img.shape)
    for i in range(depth_map.shape[0]):
        for j in range(depth_map.shape[1]):
            focused[i,j,:] = imgs[depth_map[i,j]][i,j,:]
    cv2.imwrite(os.path.join(results_path, 'focused_%s.png'%(dataset)), focused)
    np.save(os.path.join(results_path, 'focus_%s.npy'%(dataset)), focus)

(512, 512)
(512, 512)


# graph-cuts and weighted median filter

In [6]:
import gco

In [ ]:
def graph_cut()

In [ ]:
images_path = 'dataset/aligned'
results_path = 'dataset/graphcut'
if not os.path.isdir(results_path):
        os.makedirs(results_path)
datasets = ['boxes', 'cotton']

for dataset in datasets:
    if not os.path.isdir(os.path.join(results_path, dataset)):
        os.makedirs(os.path.join(results_path, dataset))
    image_files = sorted(glob.glob(os.path.join(images_path, dataset, '*')))
    
    imgs = [cv2.imread(f) for f in image_files]
    grays = [cv2.cvtCOLOR(img, cv2.COLOR_BGR2GRAY) for img in imgs]
    focus = np.load(os.path.join('dataset', 'focus', 'focus_%s.npy'%(dataset)))



In [ ]:
np.max(focus)

59336.0

In [ ]:
len(imgs)

30

In [ ]:
n=5
ii, jj = np.meshgrid(range(n), range(n))

In [ ]:
ii.shape

(5, 5)

In [ ]:
ii

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [ ]:
jj

array([[0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4]])